In [1]:
import os
import time
import requests
from datetime import datetime, timedelta
from dotenv import load_dotenv
import importlib
import googlemaps
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain.tools import Tool
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain_ollama import ChatOllama
from amadeus import Client as AmadeusClient
from langchain_community.agent_toolkits.amadeus.toolkit import AmadeusToolkit

In [2]:
load_dotenv()

OLLAMA_MODEL          = "llama3.2"
SERPAPI_API_KEY       = os.getenv("SERPAPI_API_KEY")
AMADEUS_CLIENT_ID     = os.getenv("AMADEUS_CLIENT_ID")
AMADEUS_CLIENT_SECRET = os.getenv("AMADEUS_CLIENT_SECRET")
CURRENCY_API_KEY      = os.getenv("CURRENCY_API_KEY")
GOOGLE_MAPS_API_KEY   = os.getenv("GOOGLE_MAPS_API_KEY")
OPENWEATHER_API_KEY   = os.getenv("OPENWEATHER_API_KEY")


In [4]:
from amadeus import Client
from datetime import datetime


amadeus = Client(
    client_id=AMADEUS_CLIENT_ID,
    client_secret=AMADEUS_CLIENT_SECRET
)

def search_flights_clean(origin, destination, depart_date, return_date=None, currency="USD"):
    """
    Searches flights on Amadeus and returns a simplified structure.
    """
    try:
        params = {
            "originLocationCode": origin,
            "destinationLocationCode": destination,
            "departureDate": depart_date,
            "adults": 1,
            "currencyCode": currency,
            "max": 5
        }
        if return_date:
            params["returnDate"] = return_date

        response = amadeus.shopping.flight_offers_search.get(**params)
        data = response.data

        clean_results = []
        for offer in data:
            itinerary_list = []
            for itinerary in offer["itineraries"]:
                legs = []
                for seg in itinerary["segments"]:
                    legs.append({
                        "from": seg["departure"]["iataCode"],
                        "to": seg["arrival"]["iataCode"],
                        "departure": seg["departure"]["at"],
                        "arrival": seg["arrival"]["at"],
                        "duration": seg["duration"]
                    })
                itinerary_list.append(legs)

            clean_results.append({
                "price": f"{offer['price']['total']} {offer['price']['currency']}",
                "airline": offer["validatingAirlineCodes"][0],
                "itineraries": itinerary_list
            })

        return clean_results

    except Exception as e:
        print("[Error]", e)
        return []

# --- Example ---
if __name__ == "__main__":
    results = search_flights_clean(
        origin="DEL",
        destination="CDG",
        depart_date="2025-08-28",
        return_date="2025-09-03",
        currency="EUR"
    )
    from pprint import pprint
    pprint(results)


[{'airline': 'UL',
  'itineraries': [[{'arrival': '2025-08-28T22:20:00',
                    'departure': '2025-08-28T18:45:00',
                    'duration': 'PT3H35M',
                    'from': 'DEL',
                    'to': 'CMB'},
                   {'arrival': '2025-08-29T10:55:00',
                    'departure': '2025-08-29T03:25:00',
                    'duration': 'PT11H',
                    'from': 'CMB',
                    'to': 'CDG'}],
                  [{'arrival': '2025-09-04T04:35:00',
                    'departure': '2025-09-03T12:25:00',
                    'duration': 'PT12H40M',
                    'from': 'CDG',
                    'to': 'CMB'},
                   {'arrival': '2025-09-04T17:45:00',
                    'departure': '2025-09-04T14:10:00',
                    'duration': 'PT3H35M',
                    'from': 'CMB',
                    'to': 'DEL'}]],
  'price': '621.67 EUR'},
 {'airline': 'UL',
  'itineraries': [[{'arrival': '2025-08-28T22:

In [6]:
from amadeus import Client
from datetime import datetime
import re

amadeus = Client(
    client_id=AMADEUS_CLIENT_ID,
    client_secret=AMADEUS_CLIENT_SECRET
)

def format_duration(iso_duration):
    """Convert PT8H40M → 8h 40m"""
    match = re.match(r"PT(?:(\d+)H)?(?:(\d+)M)?", iso_duration)
    if not match:
        return iso_duration
    hours = match.group(1) or "0"
    minutes = match.group(2) or "0"
    return f"{int(hours)}h {int(minutes)}m"

def format_time(iso_time):
    """Convert 2025-08-15T10:30:00 → 2025-08-15 10:30"""
    try:
        return datetime.fromisoformat(iso_time).strftime("%Y-%m-%d %H:%M")
    except:
        return iso_time

def format_segments(segments):
    """Return a string showing each leg in the trip."""
    legs = []
    for seg in segments:
        dep_airport = seg['departure']['iataCode']
        arr_airport = seg['arrival']['iataCode']
        dep_time = format_time(seg['departure']['at'])
        arr_time = format_time(seg['arrival']['at'])
        duration = format_duration(seg['duration'])
        legs.append(f"{dep_airport} ({dep_time}) → {arr_airport} ({arr_time}) [{duration}]")
    return " | ".join(legs)

def search_flights_formatted(origin, destination, depart_date, return_date=None, currency="USD"):
    try:
        params = {
            "originLocationCode": origin,
            "destinationLocationCode": destination,
            "departureDate": depart_date,
            "adults": 1,
            "currencyCode": currency,
            "max": 5
        }
        if return_date:
            params["returnDate"] = return_date

        response = amadeus.shopping.flight_offers_search.get(**params)
        data = response.data

        for offer in data:
            price = f"{offer['price']['total']} {offer['price']['currency']}"
            airline = offer["validatingAirlineCodes"][0]

            outbound_segments = offer["itineraries"][0]["segments"]
            outbound_str = format_segments(outbound_segments)

            return_str = ""
            if len(offer["itineraries"]) > 1:
                return_segments = offer["itineraries"][1]["segments"]
                return_str = format_segments(return_segments)

            print(f"Price: {price}")
            print(f"  Outbound ({airline}): {outbound_str}")
            if return_str:
                print(f"  Return ({airline}): {return_str}")
            print()

    except Exception as e:
        print("[Error]", e)

# --- Example ---
if __name__ == "__main__":
    search_flights_formatted(
        origin="DEL",
        destination="CDG",
        depart_date="2025-09-01",
        return_date="2025-09-10",
        currency="EUR"
    )


Price: 519.59 EUR
  Outbound (AZ): DEL (2025-09-01 03:00) → FCO (2025-09-01 08:10) [8h 40m] | FCO (2025-09-01 11:00) → CDG (2025-09-01 13:15) [2h 15m]
  Return (AZ): CDG (2025-09-10 06:10) → FCO (2025-09-10 08:10) [2h 0m] | FCO (2025-09-10 13:55) → DEL (2025-09-11 01:10) [7h 45m]

Price: 519.59 EUR
  Outbound (AZ): DEL (2025-09-01 03:00) → FCO (2025-09-01 08:10) [8h 40m] | FCO (2025-09-01 11:00) → CDG (2025-09-01 13:15) [2h 15m]
  Return (AZ): CDG (2025-09-10 18:15) → FCO (2025-09-10 20:25) [2h 10m] | FCO (2025-09-11 13:55) → DEL (2025-09-12 01:10) [7h 45m]

Price: 519.59 EUR
  Outbound (AZ): DEL (2025-09-01 03:00) → FCO (2025-09-01 08:10) [8h 40m] | FCO (2025-09-01 15:10) → CDG (2025-09-01 17:20) [2h 10m]
  Return (AZ): CDG (2025-09-10 06:10) → FCO (2025-09-10 08:10) [2h 0m] | FCO (2025-09-10 13:55) → DEL (2025-09-11 01:10) [7h 45m]

Price: 519.59 EUR
  Outbound (AZ): DEL (2025-09-01 03:00) → FCO (2025-09-01 08:10) [8h 40m] | FCO (2025-09-01 15:10) → CDG (2025-09-01 17:20) [2h 10m]
  

In [8]:
import requests

CURRENCY_API_URL = "https://api.currencyapi.com/v3/latest"

def convert_to_inr(amount, from_currency):
    params = {
        "apikey": CURRENCY_API_KEY,
        "base_currency": from_currency,
        "currencies": "INR"
    }
    data = requests.get(CURRENCY_API_URL, params=params).json()
    rate = data["data"]["INR"]["value"]
    return round(amount * rate, 2)

print(convert_to_inr(100, "USD"))


8741.01


In [5]:
import googlemaps

# GOOGLE_MAPS_API_KEY = "your_gmaps_key"
gmaps = googlemaps.Client(key=GOOGLE_MAPS_API_KEY)

def fetch_sightseeing(city):
    results = gmaps.places(query=f"tourist attractions in {city}").get("results", [])[:5]
    return [{"name": p["name"], "address": p.get("formatted_address"), "rating": p.get("rating")} for p in results]

print(fetch_sightseeing("valencia"))


[{'name': 'Valencia Cathedral', 'address': "Pl. de l'Almoina, s/n, Ciutat Vella, 46003 València, Valencia, Spain", 'rating': 4.6}, {'name': 'La Lonja de la Seda de Valencia', 'address': 'C/ de la Llotja, 2, Ciutat Vella, 46001 València, Valencia, Spain', 'rating': 4.6}, {'name': 'Bioparc Valencia', 'address': 'Av. Pío Baroja, 3, Campanar, 46015 València, Valencia, Spain', 'rating': 4.6}, {'name': 'El Micalet', 'address': 'Pça. de la Reina, s/n, Ciutat Vella, 46001 València, Valencia, Spain', 'rating': 4.6}, {'name': 'Torres de Serranos', 'address': 'C. de la Blanqueria, 1, Ciutat Vella, 46003 València, Valencia, Spain', 'rating': 4.6}]


In [13]:
from serpapi import GoogleSearch
import os

def run(city: str) -> str:
    print(f"[DEBUG] FetchSightseeingTool executed with city={city}")
    try:
        params = {
            "engine": "google_maps",
            "q": f"tourist attractions in {city}",
            "type": "search",
            "api_key": os.getenv("SERPAPI_API_KEY"),
        }
        search = GoogleSearch(params)
        results = search.get_dict()

        places = results.get("local_results", []) or results.get("places_results", [])  # try both keys

        if not places:
            return f"No sightseeing places found for {city}."

        attractions = []
        for p in places[:10]:
            attractions.append(
                f"- {p.get('title', 'Unknown')} "
                f"(Rating: {p.get('rating', 'N/A')}, "
                f"Address: {p.get('address', 'N/A')})"
            )

        return f"Top sightseeing attractions in {city}:\n" + "\n".join(attractions)
    except Exception as e:
        return f"[ERROR] FetchSightseeingTool failed: {str(e)}"


print(run("rome"))


[DEBUG] FetchSightseeingTool executed with city=rome


Top sightseeing attractions in rome:
- Roman Forum (Rating: 4.7, Address: 00186 Roma RM, Italy)
- Trevi Fountain (Rating: 4.7, Address: Piazza di Trevi, 00187 Roma RM, Italy)
- Pantheon (Rating: 4.8, Address: Piazza della Rotonda, 00186 Roma RM, Italy)
- Colosseum (Rating: 4.7, Address: Piazza del Colosseo, 1, 00184 Roma RM, Italy)
- Sistine Chapel (Rating: 4.7, Address: 00120 Città del Vaticano, Vatican City)
- Piazza del Popolo (Rating: 4.7, Address: 00187 Roma RM, Italy)
- Piazza Navona (Rating: 4.7, Address: 00186 Roma RM, Italy)
- Baths of Caracalla (Rating: 4.6, Address: Viale delle Terme di Caracalla, 00153 Roma RM, Italy)
- Spanish Steps (Rating: 4.6, Address: Piazza di Spagna, 00187 Roma RM, Italy)
- Capitoline Museums (Rating: 4.7, Address: Piazza del Campidoglio, 1, 00186 Roma RM, Italy)


In [10]:
import requests

# OPENWEATHER_API_KEY = "your_openweather_key"
WEATHER_API_URL = "https://api.openweathermap.org/data/2.5/forecast"

def fetch_weather(city):
    params = {"q": city, "appid": OPENWEATHER_API_KEY, "units": "metric"}
    data = requests.get(WEATHER_API_URL, params=params).json()
    daily = {}
    for entry in data.get("list", []):
        date = entry["dt_txt"].split(" ")[0]
        temps = daily.setdefault(date, [])
        temps.append(entry["main"]["temp"])
    return {d: {"min": min(t), "max": max(t)} for d, t in daily.items()}

print(fetch_weather("Paris"))


{'2025-08-09': {'min': 23.09, 'max': 23.09}, '2025-08-10': {'min': 13.61, 'max': 30.16}, '2025-08-11': {'min': 20.33, 'max': 33.84}, '2025-08-12': {'min': 23.03, 'max': 37.84}, '2025-08-13': {'min': 24.5, 'max': 31.25}, '2025-08-14': {'min': 22.55, 'max': 35.13}}


In [10]:
def run(city: str, start_date: str, end_date: str):
        params = {"q": city, "appid": OPENWEATHER_API_KEY, "units": "metric"}
        data = requests.get("https://api.openweathermap.org/data/2.5/forecast", params=params).json()

        start_dt = datetime.strptime(start_date, "%Y-%m-%d").date()
        end_dt = datetime.strptime(end_date, "%Y-%m-%d").date()
        
        filtered_forecast = {}
        for entry in data.get("list", []):
            entry_dt = datetime.strptime(entry["dt_txt"], "%Y-%m-%d %H:%M:%S")
            entry_date = entry_dt.date()

            if start_dt <= entry_date <= end_dt:
                day_str = entry_date.strftime("%Y-%m-%d")
                filtered_forecast.setdefault(day_str, []).append({
                    "time": entry_dt.strftime("%H:%M"),
                    "temp": round(entry["main"]["temp"], 1),
                    "feels_like": round(entry["main"]["feels_like"], 1),
                    "condition": entry["weather"][0]["description"].title(),
                    "humidity": entry["main"]["humidity"],
                    "wind_speed": entry["wind"]["speed"]
                })

        return filtered_forecast

forecast = run("tokyo","2025-08-15","2025-08-20")
print(forecast)

{'2025-08-15': [{'time': '00:00', 'temp': 29.0, 'feels_like': 31.5, 'condition': 'Scattered Clouds', 'humidity': 63, 'wind_speed': 3.03}, {'time': '03:00', 'temp': 31.7, 'feels_like': 34.3, 'condition': 'Few Clouds', 'humidity': 52, 'wind_speed': 5.33}, {'time': '06:00', 'temp': 32.5, 'feels_like': 34.4, 'condition': 'Few Clouds', 'humidity': 46, 'wind_speed': 6.27}, {'time': '09:00', 'temp': 29.7, 'feels_like': 31.5, 'condition': 'Few Clouds', 'humidity': 55, 'wind_speed': 7.95}, {'time': '12:00', 'temp': 28.1, 'feels_like': 30.2, 'condition': 'Few Clouds', 'humidity': 64, 'wind_speed': 6.95}, {'time': '15:00', 'temp': 27.6, 'feels_like': 29.8, 'condition': 'Clear Sky', 'humidity': 68, 'wind_speed': 4.88}, {'time': '18:00', 'temp': 27.1, 'feels_like': 29.2, 'condition': 'Clear Sky', 'humidity': 72, 'wind_speed': 4.59}, {'time': '21:00', 'temp': 27.1, 'feels_like': 29.1, 'condition': 'Few Clouds', 'humidity': 72, 'wind_speed': 3.64}], '2025-08-16': [{'time': '00:00', 'temp': 29.4, 'fee

In [17]:
from serpapi import GoogleSearch

params = {
    "q": "Best hotels in Paris",
    "api_key": SERPAPI_API_KEY,
    "engine": "google"
}

search = GoogleSearch(params)
results = search.get_dict().get("organic_results", [])
print([r.get("link") for r in results[:5]])


['https://www.reddit.com/r/chubbytravel/comments/1d9k9l1/best_hotel_in_paris_theres_so_many_to_choose_from/', 'https://www.cntraveler.com/gallery/the-best-hotels-in-paris', 'https://everydayparisian.com/where-to-stay-in-paris/', 'https://www.forbestravelguide.com/destinations/paris-france', 'https://www.tripadvisor.com/Hotels-g187147-Paris_Ile_de_France-Hotels.html']


In [13]:
import requests

def fetch_html(url):
    r = requests.get(url, timeout=10)
    r.raise_for_status()
    return r.text

print(fetch_html("https://example.com")[:500])  # Print first 500 chars


<!doctype html>
<html>
<head>
    <title>Example Domain</title>

    <meta charset="utf-8" />
    <meta http-equiv="Content-type" content="text/html; charset=utf-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1" />
    <style type="text/css">
    body {
        background-color: #f0f0f2;
        margin: 0;
        padding: 0;
        font-family: -apple-system, system-ui, BlinkMacSystemFont, "Segoe UI", "Open Sans", "Helvetica Neue", Helvetica, Arial, sans-serif;
    


In [1]:
from langchain_ollama import ChatOllama
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

OLLAMA_MODEL = "llama3.2"
llm = ChatOllama(model=OLLAMA_MODEL)

def parse_html_with_llm(html, schema):
    prompt = PromptTemplate(
        input_variables=["html", "schema"],
        template="""
Extract structured data from the following HTML according to this JSON schema:

HTML:
{html}

Schema:
{schema}

Return only valid JSON matching the schema.
"""
    )
    chain = LLMChain(llm=llm, prompt=prompt)
    return chain.run({"html": html, "schema": schema})

html = "<h1>Hotel Name: Paris Luxury Stay</h1><p>Price: $200</p>"
schema = '{"hotel_name": "string", "price_usd": "number"}'
print(parse_html_with_llm(html, schema))


/tmp/ipykernel_2899/1379746661.py:23: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)
/tmp/ipykernel_2899/1379746661.py:24: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return chain.run({"html": html, "schema": schema})


To extract structured data from the HTML according to the provided JSON schema, we can use Python with the BeautifulSoup and jsonschema libraries.

Firstly, install the necessary libraries by running:

```bash
pip install beautifulsoup4 jsonschema
```

Then, write a Python script that reads the HTML file (replace 'html_file.html' with your actual file name), extracts relevant data, and validates it against the provided JSON schema:

```python
import json
from bs4 import BeautifulSoup

# Function to extract hotel information from the given HTML content
def extract_hotel_info(html_content):
    soup = BeautifulSoup(html_content, "html.parser")
    
    # Extracting hotel_name
    hotel_name_element = soup.find('h1')
    if not hotel_name_element:
        return None
    
    # Extracting price in USD
    price_element = soup.find('p', attrs={'class': 'price-usd'})
    if price_element is None:
        return None

    hotel_name = hotel_name_element.text.strip()
    price_usd = float(pri

In [18]:
def format_flight_data(raw_data):
    flights_summary = []
    for offer in raw_data:
        price = offer["price"]["grandTotal"]
        currency = offer["price"]["currency"]
        itineraries = []
        for itin in offer["itineraries"]:
            duration = itin["duration"]
            segments = []
            for seg in itin["segments"]:
                segments.append({
                    "from": seg["departure"]["iataCode"],
                    "to": seg["arrival"]["iataCode"],
                    "dep_time": seg["departure"]["at"],
                    "arr_time": seg["arrival"]["at"],
                    "carrier": seg["carrierCode"],
                    "flight_no": seg["number"],
                    "duration": seg["duration"]
                })
            itineraries.append({"duration": duration, "segments": segments})
        flights_summary.append({
            "price": f"{price} {currency}",
            "itineraries": itineraries
        })
    return flights_summary

# Example usage:
pretty_flights = format_flight_data(resp.data)
for f in pretty_flights:
    print(f"Price: {f['price']}")
    for itin in f['itineraries']:
        print(f"  Total Duration: {itin['duration']}")
        for seg in itin['segments']:
            print(f"    {seg['from']} → {seg['to']} | {seg['dep_time']} → {seg['arr_time']} | {seg['carrier']} {seg['flight_no']}")
    print()


Price: 272.44 EUR
  Total Duration: PT12H30M
    DEL → DXB | 2025-09-01T11:00:00 → 2025-09-01T13:00:00 | EK 511
    DXB → CDG | 2025-09-01T14:40:00 → 2025-09-01T20:00:00 | EK 75

Price: 272.44 EUR
  Total Duration: PT12H45M
    DEL → DXB | 2025-09-01T04:15:00 → 2025-09-01T06:20:00 | EK 513
    DXB → CDG | 2025-09-01T08:20:00 → 2025-09-01T13:30:00 | EK 73

Price: 272.44 EUR
  Total Duration: PT14H55M
    DEL → DXB | 2025-09-01T22:00:00 → 2025-09-02T00:05:00 | EK 515
    DXB → CDG | 2025-09-02T04:05:00 → 2025-09-02T09:25:00 | EK 71

Price: 272.44 EUR
  Total Duration: PT19H
    DEL → DXB | 2025-09-01T22:00:00 → 2025-09-02T00:05:00 | EK 515
    DXB → CDG | 2025-09-02T08:20:00 → 2025-09-02T13:30:00 | EK 73

Price: 272.44 EUR
  Total Duration: PT19H15M
    DEL → DXB | 2025-09-01T04:15:00 → 2025-09-01T06:20:00 | EK 513
    DXB → CDG | 2025-09-01T14:40:00 → 2025-09-01T20:00:00 | EK 75

Price: 272.44 EUR
  Total Duration: PT20H35M
    DEL → DXB | 2025-09-01T16:20:00 → 2025-09-01T18:20:00 | EK 

In [1]:
import os
import json
import requests
from langchain.tools import BaseTool
from typing import Optional, Type, List, Dict
from pydantic import BaseModel, Field
from dotenv import load_dotenv
from datetime import datetime, timedelta
import googlemaps
import time

In [2]:
from serpapi import GoogleSearch

def get_round_trip_flights(departure_id, arrival_id, outbound_date, return_date, currency="USD", hl="en", api_key=None):
    """
    Fetches round-trip flight results from SerpAPI Google Flights engine
    following documentation's requirement for `departure_token` on return legs.
    """

    # --- STEP 1: Outbound search ---
    params_outbound = {
        "engine": "google_flights",
        "departure_id": departure_id,
        "arrival_id": arrival_id,
        "outbound_date": outbound_date,
        "return_date": return_date,
        "currency": currency,
        "hl": hl,
        "type": "1",  # round trip
        "api_key": api_key
    }

    search_outbound = GoogleSearch(params_outbound)
    results_outbound = search_outbound.get_dict()

    if "best_flights" not in results_outbound or not results_outbound["best_flights"]:
        print("No outbound flights found.")
        return []

    round_trip_results = []

    for outbound_option in results_outbound["best_flights"]:
        departure_token = outbound_option.get("departure_token")
        if not departure_token:
            continue

        # --- STEP 2: Return search using departure_token ---
        params_return = {
            "engine": "google_flights",
            "departure_token": departure_token,
            "api_key": api_key
        }
        search_return = GoogleSearch(params_return)
        results_return = search_return.get_dict()

        return_flights = results_return.get("best_flights", [])

        # Store both legs together
        round_trip_results.append({
            "price": outbound_option.get("price"),
            "currency": currency,
            "outbound": outbound_option,
            "return": return_flights[0] if return_flights else None
        })

    return round_trip_results

In [59]:
# result =get_round_trip("DEL", "CDG", "2025-08-15", "2025-08-18")
# print(result)
result = get_round_trip_flights("DEL", "CDG", "2025-08-15", "2025-08-18","USD" , SERPAPI_API_KEY)
print(result)

No outbound flights found.
[]


In [55]:
from datetime import datetime

def print_flight_results(results, currency="USD"):
    for flight in results:
        outbound = flight.get("outbound")
        ret = flight.get("return")

        # If no outbound flight, skip
        if not outbound:
            continue

        # Calculate total duration
        total_minutes = outbound.get("duration", 0) + (ret.get("duration", 0) if ret else 0)
        hours, minutes = divmod(total_minutes, 60)
        total_duration_str = f"PT{hours}H{minutes}M"

        # Print price and duration
        print(f"Price: {flight['price']} {currency}")
        print(f"  Total Duration: {total_duration_str}")

        # Outbound
        dep_time = datetime.strptime(outbound['departure_airport']['time'], "%Y-%m-%d %H:%M").isoformat()
        arr_time = datetime.strptime(outbound['arrival_airport']['time'], "%Y-%m-%d %H:%M").isoformat()
        print(f"    {outbound['departure_airport']['id']} → {outbound['arrival_airport']['id']} | {dep_time} → {arr_time} | {outbound['flight_number']}")

        # outbound = flight.get("outbound")
        # if outbound:
        #     print("  Outbound Flight:")
        #     print(f"    {outbound['departure_airport']['id']} → {outbound['arrival_airport']['id']} "
        #           f"| {outbound['departure_airport']['time']} → {outbound['arrival_airport']['time']} "
        #           f"| {outbound['flight_number']} ({outbound['airline']})")
                  
        # Return (if available)
        rtn = flight.get("return")
        if rtn:
            print("  Return Flight:")
            print(f"    {rtn['departure_airport']['id']} → {rtn['arrival_airport']['id']} "
                  f"| {rtn['departure_airport']['time']} → {rtn['arrival_airport']['time']} "
                  f"| {rtn['flight_number']} ({rtn['airline']})")

        print()  # Blank line between flights


In [56]:
print_flight_results(result, currency="USD")

Price: 814 USD
  Total Duration: PT11H5M
    DEL → AUH | 2025-08-15T21:15:00 → 2025-08-15T23:20:00 | EY 217
  Return Flight:
    AUH → CDG | 2025-08-16 02:30 → 2025-08-16 08:00 | EY 31 (Etihad)

Price: 841 USD
  Total Duration: PT10H55M
    DEL → LHR | 2025-08-15T01:35:00 → 2025-08-15T06:40:00 | BA 142
  Return Flight:
    LHR → CDG | 2025-08-15 08:40 → 2025-08-15 11:00 | BA 306 (British Airways)

Price: 941 USD
  Total Duration: PT10H55M
    DEL → MCT | 2025-08-15T08:55:00 → 2025-08-15T10:45:00 | WY 242
  Return Flight:
    MCT → CDG | 2025-08-15 14:10 → 2025-08-15 19:45 | WY 131 (Oman Air)

Price: 1155 USD
  Total Duration: PT9H35M
    DEL → CDG | 2025-08-15T00:50:00 → 2025-08-15T06:55:00 | AF 225



In [18]:
import json

def format_flights(out, as_json=False, top_n=5):
    flights_data = []
    for f in out[:top_n]:  # limit to first N options
        flight_info = {
            "price_usd": f.get("price", "N/A"),
            "legs": []
        }

        for i, leg in enumerate(f.get("legs", [])):
            direction = "Outbound" if i == 0 else "Return"
            dep_airport = leg.get("departure_airport", {}).get("code", "N/A")
            arr_airport = leg.get("arrival_airport", {}).get("code", "N/A")
            dep_time = leg.get("departure_time", "N/A")
            arr_time = leg.get("arrival_time", "N/A")

            airline = leg.get("airline", "Unknown Airline")
            airline_name = airline.get("name") if isinstance(airline, dict) else airline

            leg_info = {
                "direction": direction,
                "from": dep_airport,
                "to": arr_airport,
                "airline": airline_name,
                "departure_time": dep_time,
                "arrival_time": arr_time
            }
            flight_info["legs"].append(leg_info)

        flights_data.append(flight_info)

    if as_json:
        return json.dumps(flights_data, indent=2)

    # Human-readable output
    lines = [f"Found {len(flights_data)} flight option(s):"]
    for f in flights_data:
        lines.append(f"\nPrice: ${f['price_usd']}")
        for leg in f["legs"]:
            lines.append(
                f"  {leg['direction']}: {leg['from']} → {leg['to']} "
                f"| {leg['airline']} | {leg['departure_time']} → {leg['arrival_time']}"
            )
    return "\n".join(lines)


In [19]:
print(format_flights(result))

Found 5 flight option(s):

Price: $2281
  Outbound: N/A → N/A | United | N/A → N/A
  Return: N/A → N/A | United | N/A → N/A

Price: $2456
  Outbound: N/A → N/A | Cathay Pacific | N/A → N/A
  Return: N/A → N/A | Cathay Pacific | N/A → N/A
  Return: N/A → N/A | American | N/A → N/A

Price: $2460
  Outbound: N/A → N/A | United | N/A → N/A
  Return: N/A → N/A | United | N/A → N/A
  Return: N/A → N/A | United | N/A → N/A

Price: $2467
  Outbound: N/A → N/A | Air China | N/A → N/A
  Return: N/A → N/A | United | N/A → N/A
  Return: N/A → N/A | United | N/A → N/A

Price: $2643
  Outbound: N/A → N/A | Asiana Airlines | N/A → N/A
  Return: N/A → N/A | Air Canada | N/A → N/A
  Return: N/A → N/A | Air Canada | N/A → N/A


In [74]:
import os
import requests

# SERPAPI_KEY = os.getenv("SERPAPI_KEY")  # Set your key in env or replace directly

def fetch_round_trip_flights(origin, destination, depart_date, return_date, currency="USD"):
    """
    Fetch outbound and return flights using SerpAPI Google Flights.
    Will try departure_token first, then fallback to separate one-way queries.
    """
    
    def get_flights(params):
        try:
            r = requests.get("https://serpapi.com/search", params=params)
            r.raise_for_status()
            return r.json()
        except requests.RequestException as e:
            print(f"[Error] {e}")
            return {}
    
    # Step 1: Initial round trip search
    params_outbound = {
        "engine": "google_flights",
        "departure_id": origin,
        "arrival_id": destination,
        "type": "1",  # round trip
        "outbound_date": depart_date,
        "return_date": return_date,
        "currency": currency,
        "api_key": SERPAPI_API_KEY
    }
    outbound_data = get_flights(params_outbound)
    
    outbound_flights = outbound_data.get("best_flights") or outbound_data.get("other_flights", [])
    departure_token = outbound_data.get("departure_token")
    
    # Step 2: Try fetching return flights with departure_token
    return_flights = []
    if departure_token:
        params_return = {
            "engine": "google_flights",
            "departure_token": departure_token,
            "outbound_date": return_date,
            "currency": currency,
            "api_key": SERPAPI_API_KEY
        }
        return_data = get_flights(params_return)
        return_flights = return_data.get("other_flights", [])
    
    # Step 3: Fallback if token missing or no return flights
    if not return_flights:
        params_return_manual = {
            "engine": "google_flights",
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": return_date,
            "currency": currency,
            "api_key": SERPAPI_API_KEY
        }
        return_data_manual = get_flights(params_return_manual)
        return_flights = return_data_manual.get("best_flights") or return_data_manual.get("other_flights", [])
    
    return {
        "outbound": outbound_flights,
        "return": return_flights
    }

# Example usage
if __name__ == "__main__":
    result = fetch_round_trip_flights("PEK", "AUS", "2025-08-15", "2025-08-18")
    print(result)


[Error] 400 Client Error: Bad Request for url: https://serpapi.com/search?engine=google_flights&departure_id=AUS&arrival_id=PEK&outbound_date=2025-08-18&currency=USD&api_key=d490e423f17a080996b2513e582b8514577c23a9c7940a5a872fc1a42bb4e47e
{'outbound': [{'flights': [{'departure_airport': {'name': 'Beijing Capital International Airport', 'id': 'PEK', 'time': '2025-08-15 17:25'}, 'arrival_airport': {'name': 'San Francisco International Airport', 'id': 'SFO', 'time': '2025-08-15 14:20'}, 'duration': 715, 'airplane': 'Boeing 777', 'airline': 'United', 'airline_logo': 'https://www.gstatic.com/flights/airline_logos/70px/UA.png', 'travel_class': 'Economy', 'flight_number': 'UA 889', 'legroom': '31 in', 'extensions': ['Average legroom (31 in)', 'Wi-Fi for a fee', 'In-seat power & USB outlets', 'On-demand video', 'Carbon emissions estimate: 655 kg'], 'overnight': True, 'often_delayed_by_over_30_min': True}, {'departure_airport': {'name': 'San Francisco International Airport', 'id': 'SFO', 'time'

In [ ]:
result = get_round_trip_all("PEK", "AUS", "2025-08-15", "2025-08-18")
print(result)